### Import libraries and datasets

In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.feature_selection import VarianceThreshold

In [6]:
df = pd.read_excel('PIK3CA.xlsx')

In [7]:
rdkit_df = pd.read_excel('PIK3CA_descriptors.xlsx')

In [9]:
mordred_df = pd.read_csv('PIK3CA_mordred_desc.csv')

/tmp/ipykernel_10438/3271736655.py:1: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,54,55,56,57,58,59,60,61,62,142,143,151,152,153,154,155,156,157,158,159,160,161,169,170,178,179,187,188,196,197,205,206,214,215,223,224,232,233,261,262,263,264,265,266,267,268,269,349,350,358,359,367,368,369,370,371,372,373,374,375,376,377,385,386,394,395,403,404,412,413,421,422,430,431,439,440,448,449,456,457,464,465,472,473,474,475,476,477,478,479,480,481,488,489,496,497,504,505,512,513,520,521,528,529,536,537,544,545,552,553,560,561,568,569,570,571,572,573,574,575,576,577,584,585,592,593,600,601,608,609,616,617,624,625,632,633,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,7

In [11]:
df.head()

,Unnamed: 0,molecule_chembl_id,canonical_smiles,pIC50,IC50,standard_units
0,0,CHEMBL435507,CC1CN(c2cc(=O)c3ccc4ccccc4c3o2)CCO1,5.619789,2400.0,nM
1,1,CHEMBL98350,O=c1cc(N2CCOCC2)oc2c(-c3ccccc3)cccc12,5.638272,2300.0,nM
2,2,CHEMBL104468,O=c1cc(N2CCOCC2)oc2c1ccc1ccccc12,4.886057,13000.0,nM
3,3,CHEMBL188678,O=c1cc(N2CCOCC2)oc2c(-c3cccc4c3sc3ccccc34)cccc12,5.301030,5000.0,nM
4,4,CHEMBL379156,O=C1NC(=O)/C(=C/c2ccc(-c3ccc(F)cc3O)o2)S1,6.026872,940.0,nM


### RDKit Results

In [12]:
rdkit_df.head()

,Unnamed: 0,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,13.300824,13.300824,0.373221,-3.873970,0.646725,295.338,278.202,295.120843,112,...,0,0,0,0,0,0,0,0,0,0
1,1,13.362025,13.362025,0.305310,-3.665314,0.728379,307.349,290.213,307.120843,116,...,0,0,0,0,0,0,0,0,0,0
2,2,13.168156,13.168156,0.247596,-3.587512,0.643252,281.311,266.191,281.105193,106,...,0,0,0,0,0,0,0,0,0,0
3,3,13.774156,13.774156,0.116155,-3.706189,0.374251,413.498,394.346,413.108564,148,...,0,0,0,0,0,0,0,1,0,0
4,4,13.864007,13.864007,0.010190,-1.434530,0.833356,305.286,297.222,305.015807,106,...,1,0,0,0,0,0,0,0,0,0


In [13]:
rdkit_df = rdkit_df.drop(rdkit_df.columns[0], axis=1)

In [14]:
rdkit_df.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.300824,13.300824,0.373221,-3.873970,0.646725,295.338,278.202,295.120843,112,0,...,0,0,0,0,0,0,0,0,0,0
1,13.362025,13.362025,0.305310,-3.665314,0.728379,307.349,290.213,307.120843,116,0,...,0,0,0,0,0,0,0,0,0,0
2,13.168156,13.168156,0.247596,-3.587512,0.643252,281.311,266.191,281.105193,106,0,...,0,0,0,0,0,0,0,0,0,0
3,13.774156,13.774156,0.116155,-3.706189,0.374251,413.498,394.346,413.108564,148,0,...,0,0,0,0,0,0,0,1,0,0
4,13.864007,13.864007,0.010190,-1.434530,0.833356,305.286,297.222,305.015807,106,0,...,1,0,0,0,0,0,0,0,0,0


In [19]:
len(rdkit_df)

4790

In [15]:
rdkit_df['pic50'] = df['pIC50']

In [17]:
rdkit_df.columns

Index(['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex',
       'MinEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt',
       'NumValenceElectrons', 'NumRadicalElectrons',
       ...
       'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene', 'fr_tetrazole',
       'fr_thiazole', 'fr_thiocyan', 'fr_thiophene', 'fr_unbrch_alkane',
       'fr_urea', 'pic50'],
      dtype='object', length=210)

In [20]:
rdkit2 = rdkit_df.dropna(axis=0)
len(rdkit2) # no nan values

4790

In [21]:
rdkit2 = rdkit2.reset_index()

In [22]:
y = rdkit2['pic50']
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 4790 entries, 0 to 4789
Series name: pic50
Non-Null Count  Dtype  
--------------  -----  
4790 non-null   float64
dtypes: float64(1)
memory usage: 37.5 KB


In [23]:
X = rdkit2.drop(columns=['pic50'])  
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Columns: 210 entries, index to fr_urea
dtypes: float64(103), int64(107)
memory usage: 7.7 MB


In [24]:
rdkit2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Columns: 211 entries, index to pic50
dtypes: float64(104), int64(107)
memory usage: 7.7 MB


#### (i) Feature Selection

In [26]:
is_good_col = X.isnull().mean() < 0.5
is_good_col

index                True
MaxAbsEStateIndex    True
MaxEStateIndex       True
MinAbsEStateIndex    True
MinEStateIndex       True
                     ... 
fr_thiazole          True
fr_thiocyan          True
fr_thiophene         True
fr_unbrch_alkane     True
fr_urea              True
Length: 210, dtype: bool

In [27]:
good_cols = is_good_col[is_good_col].index
X = X[good_cols]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Columns: 210 entries, index to fr_urea
dtypes: float64(103), int64(107)
memory usage: 7.7 MB


In [28]:
X.head()

,index,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,13.300824,13.300824,0.373221,-3.873970,0.646725,295.338,278.202,295.120843,112,...,0,0,0,0,0,0,0,0,0,0
1,1,13.362025,13.362025,0.305310,-3.665314,0.728379,307.349,290.213,307.120843,116,...,0,0,0,0,0,0,0,0,0,0
2,2,13.168156,13.168156,0.247596,-3.587512,0.643252,281.311,266.191,281.105193,106,...,0,0,0,0,0,0,0,0,0,0
3,3,13.774156,13.774156,0.116155,-3.706189,0.374251,413.498,394.346,413.108564,148,...,0,0,0,0,0,0,0,1,0,0
4,4,13.864007,13.864007,0.010190,-1.434530,0.833356,305.286,297.222,305.015807,106,...,1,0,0,0,0,0,0,0,0,0


In [29]:
def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

In [30]:
X = remove_low_variance(X, threshold=0.1)

In [38]:
X.info() # 136 columns are remaining only

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Columns: 136 entries, index to fr_urea
dtypes: float64(86), int64(50)
memory usage: 5.0 MB


#### Feature Selection (by correlation)

In [33]:
from scipy.stats import spearmanr

In [34]:
# Calculate the Spearman correlation matrix
correlation_matrix, _ = spearmanr(X)

In [35]:
# Create a mask for upper triangle
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

In [36]:
# Find features with correlation greater than a threshold
threshold = 0.95
high_corr_features = set()

num_features = X.shape[1]  # Number of columns in the DataFrame

for i in range(num_features):
    for j in range(i+1, num_features):  # Start from i+1 to avoid self-comparison and redundant pairs
        if abs(correlation_matrix[i, j]) > threshold:
            colname_i = X.columns[i]
            colname_j = X.columns[j]
            
            # Determine which feature has higher correlation with y
            corr_with_y_i = y.corr(X[colname_i])
            corr_with_y_j = y.corr(X[colname_j])
            
            # Remove the feature with lesser correlation with y
            if corr_with_y_i > corr_with_y_j:
                high_corr_features.add(colname_j)
            else:
                high_corr_features.add(colname_i)

In [37]:
# Drop features with lesser correlation with y
X_reduced_95 = X.drop(columns=high_corr_features)

# Print the result
print("Original DataFrame:")
print(X)
print("\nDataFrame with Redundant Features Removed:")
print(X_reduced_95)

Original DataFrame:
      index  MaxAbsEStateIndex  MaxEStateIndex  MinEStateIndex     MolWt  \
0         0          13.300824       13.300824       -3.873970   295.338   
1         1          13.362025       13.362025       -3.665314   307.349   
2         2          13.168156       13.168156       -3.587512   281.311   
3         3          13.774156       13.774156       -3.706189   413.498   
4         4          13.864007       13.864007       -1.434530   305.286   
...     ...                ...             ...             ...       ...   
4785   4785          15.670877       15.670877       -5.865303  1093.295   
4786   4786           9.121117        9.121117       -3.868042   364.409   
4787   4787           9.096410        9.096410       -3.809673   384.827   
4788   4788          12.893909       12.893909       -4.211548   425.880   
4789   4789          14.913980       14.913980       -4.271861   564.031   

      HeavyAtomMolWt   ExactMolWt  NumValenceElectrons  BCUT2D_MWHI

#### Lazy Predict

In [39]:
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [40]:
# Split the dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_reduced_95, y, test_size=0.2, random_state=50)

In [41]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

 98%|█████████████████████████████████████████▉ | 41/42 [50:06<00:56, 56.89s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11575
[LightGBM] [Info] Number of data points in the train set: 3832, number of used features: 106
[LightGBM] [Info] Start training from score 6.795915


100%|███████████████████████████████████████████| 42/42 [50:06<00:00, 71.59s/it]


In [44]:
print(models)

                               Adjusted R-Squared        R-Squared      RMSE  \
Model                                                                          
LGBMRegressor                                0.67             0.71      0.64   
ExtraTreesRegressor                          0.67             0.71      0.64   
SVR                                          0.66             0.70      0.64   
HistGradientBoostingRegressor                0.66             0.70      0.64   
NuSVR                                        0.66             0.70      0.65   
XGBRegressor                                 0.65             0.69      0.65   
KNeighborsRegressor                          0.65             0.69      0.65   
RandomForestRegressor                        0.65             0.69      0.66   
MLPRegressor                                 0.61             0.66      0.69   
BaggingRegressor                             0.60             0.64      0.70   
GradientBoostingRegressor               